In [1]:
import pandas as pd
from datetime import datetime, timedelta

def generate_cashflow(face_value, coupon_rate, issue_date, maturity_date, frequency=1):
    """Generates a cashflow table for the bond."""
    cashflows = []
    current_date = datetime.strptime(issue_date, "%Y-%m-%d")
    maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
    
    while current_date <= maturity_date:
        days_in_period = 366 if ((current_date.year % 4 == 0 and current_date.year % 100 != 0) or (current_date.year % 400 == 0)) else 365
        interest_payment = (coupon_rate * face_value * days_in_period) / 365  # Adjusting for leap years
        
        cashflows.append({
            "Date": current_date.strftime("%Y-%m-%d"),
            "Cashflow": interest_payment
        })
        
        current_date = datetime(current_date.year + 1, current_date.month, current_date.day)
    
    # Add face value repayment at maturity
    cashflows[-1]["Cashflow"] += face_value
    
    df = pd.DataFrame(cashflows)
    print("Generated Cashflows:")
    print(df)
    return df

def calculate_buy_price(cashflows, buy_date, coupon_rate, face_value):
    """Calculates the buy price using present value of future cashflows."""
    buy_date = datetime.strptime(buy_date, "%Y-%m-%d")
    discount_rate = coupon_rate - 0.002  # Using coupon rate - 0.2% as discount rate
    total_present_value = 0
    
    for _, row in cashflows.iterrows():
        cashflow_date = datetime.strptime(row["Date"], "%Y-%m-%d")
        if cashflow_date > buy_date:
            time_diff = (cashflow_date - buy_date).days / 365
            discount_factor = (1 + discount_rate) ** time_diff
            present_value = row["Cashflow"] / discount_factor
            total_present_value += present_value
            print(f"Discounting {row['Cashflow']} on {cashflow_date.strftime('%Y-%m-%d')} with factor {discount_factor:.6f}: {present_value:.2f}")
    
    print(f"Total Present Value of Cashflows: {total_present_value:.2f}")
    buy_price = total_present_value / face_value * 100  # Convert to price per 100 face value
    print(f"Computed Buy Price: {buy_price:.2f}")
    return buy_price

def calculate_sell_price(buy_price, tax_rate, transaction_fee, customer_type, sell_date, resale_rate, cashflows, face_value):
    """Calculates the sell price based on the provided factors."""
    sell_date = datetime.strptime(sell_date, "%Y-%m-%d")
    sell_cashflows = cashflows[cashflows["Date"] > sell_date.strftime("%Y-%m-%d")]
    
    total_sell_value = 0
    for _, row in sell_cashflows.iterrows():
        cashflow_date = datetime.strptime(row["Date"], "%Y-%m-%d")
        time_diff = (cashflow_date - sell_date).days / 365
        discount_factor = (1 + resale_rate) ** time_diff
        present_value = row["Cashflow"] / discount_factor
        total_sell_value += present_value
        print(f"Discounting {row['Cashflow']} on {cashflow_date.strftime('%Y-%m-%d')} with factor {discount_factor:.6f}: {present_value:.2f}")
    
    print(f"Total Present Value of Sell Cashflows: {total_sell_value:.2f}")
    sell_price = total_sell_value / face_value * 100  # Convert to price per 100 face value
    
    # Apply tax and transaction fees
    if customer_type == "Individual":
        sell_price *= (1 - tax_rate)  # Apply tax for individuals
    
    sell_price -= transaction_fee * buy_price  # Apply transaction fee
    print(f"Computed Sell Price: {sell_price:.2f}")
    return sell_price

def calculate_total_received(sell_price, coupon_received, tax_rate, customer_type, face_value):
    """Calculates the total amount received after tax deductions."""
    if customer_type == "Individual":
        coupon_received *= (1 - tax_rate)  # Apply tax on coupon payments
    total_received = sell_price * face_value / 100 + coupon_received
    print(f"Total Cash Received Calculation: {sell_price:.2f} * {face_value} / 100 + {coupon_received:.2f} = {total_received:.2f}")
    return total_received

# Example usage
face_value = 100000
coupon_rate = 0.105
issue_date = "2021-03-30"
maturity_date = "2024-03-30"
yield_rate = 0.11
tax_rate = 0.05  # 5% tax on coupon
transaction_fee = 0.01  # 1% transaction fee
customer_type = "Institutional"
resale_rate = 0.11

cashflows = generate_cashflow(face_value, coupon_rate, issue_date, maturity_date)
buy_price = calculate_buy_price(cashflows, "2021-04-26", coupon_rate, face_value)
sell_price = calculate_sell_price(buy_price, tax_rate, transaction_fee, customer_type, "2021-05-07", resale_rate, cashflows, face_value)
coupon_received = coupon_rate * face_value  # Simplified coupon calculation
total_received = calculate_total_received(sell_price, coupon_received, tax_rate, customer_type, face_value)

print(f"Final Buy Price: {buy_price:.2f}")
print(f"Final Sell Price: {sell_price:.2f}")
print(f"Final Total Cash Received: {total_received:.2f}")


Generated Cashflows:
         Date       Cashflow
0  2021-03-30   10500.000000
1  2022-03-30   10500.000000
2  2023-03-30   10500.000000
3  2024-03-30  110528.767123
Discounting 10500.0 on 2022-03-30 with factor 1.095030: 9588.78
Discounting 10500.0 on 2023-03-30 with factor 1.207818: 8693.36
Discounting 110528.76712328767 on 2024-03-30 with factor 1.332581: 82943.35
Total Present Value of Cashflows: 101225.49
Computed Buy Price: 101.23
Discounting 10500.0 on 2022-03-30 with factor 1.098005: 9562.80
Discounting 10500.0 on 2023-03-30 with factor 1.218786: 8615.13
Discounting 110528.76712328767 on 2024-03-30 with factor 1.353239: 81677.19
Total Present Value of Sell Cashflows: 99855.11
Computed Sell Price: 98.84
Total Cash Received Calculation: 98.84 * 100000 / 100 + 10500.00 = 109342.86
Final Buy Price: 101.23
Final Sell Price: 98.84
Final Total Cash Received: 109342.86
